In [61]:
# If first time running, install and/or update opentrons package
import sys
!{sys.executable} -m pip install opentrons
!{sys.executable} -m pip install --upgrade opentrons

In [83]:
# Tutorial example for simulating protocols 
from opentrons import simulate

# Initializing simulation to API version used
protocol_tutorial = simulate.get_protocol_api('2.10')
# Homing simulated liquid handler machine
protocol_tutorial.home()

# Labware
# well plate at slot 2
plate = protocol_tutorial.load_labware('corning_96_wellplate_360ul_flat', '2')
# tiprack at slot 1
tiprack = protocol_tutorial.load_labware('opentrons_96_tiprack_300ul', '1')

# Pipettes
left_pipette = protocol_tutorial.load_instrument('p300_multi', 'left', tip_racks=[tiprack])

# Commands
def run(protocol: protocol_api.ProtocolContext):
    left_pipette.transfer(100, plate['A1'], plate['A2'])

run(protocol)
for line in protocol_tutorial.commands(): 
    print(line)

C:\SPB_Data\.opentrons\robot_settings.json not found. Loading defaults
C:\SPB_Data\.opentrons\deck_calibration.json not found. Loading defaults


Transferring 100.0 from A1 of Corning 96 Well Plate 360 µL Flat on 2 to A2 of Corning 96 Well Plate 360 µL Flat on 2
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 1
Aspirating 100.0 uL from A1 of Corning 96 Well Plate 360 µL Flat on 2 at 150.0 uL/sec
Dispensing 100.0 uL into A2 of Corning 96 Well Plate 360 µL Flat on 2 at 300.0 uL/sec
Dropping tip into A1 of Opentrons Fixed Trash on 12


In [1]:
# Imported protocol adapted for simulation
# Protocol name: "PCR Plate Prep with 384 Well Plate"
# Link to page: https://protocols.opentrons.com/protocol/13ea1e-part2

from opentrons import simulate
import json
import math

# Adapt protocol to Jupyter Simulation
ctx = simulate.get_protocol_api('2.9')
ctx.home()

# Importing definitions of custom labware
with open('PCR_prep/custom_labware/Nest 2.2mL 96 well plate.json') as labware_file:
    labware_def_n_2ml_96_wp = json.load(labware_file)
    # = ctx.load_labware_from_definition(labware_def_n_2ml_96_wp,   )
with open('PCR_prep/custom_labware/Opentrons 24 Tube Rack 1500 µL.json') as labware_file:
    labware_def_ot_24_tr_1500ul = json.load(labware_file)
    # = ctx.load_labware_from_definition(labware_def_ot_24_tr_1500ul,   )
with open('PCR_prep/custom_labware/Pr1ma 384 Well Plate 50 µL.json') as labware_file:
    labware_def_pr_384_wp_50ul = json.load(labware_file)
    # = ctx.load_labware_from_definition(labware_def_pr_384_wp_50ul,   )


def get_values(*names):
    #import json
    _all_values = json.loads("""{"num_samp":96,"overage_percent":10,"mix_reps":15,"p20_mount":"right","p300_mount":"left"}""")
    return [_all_values[n] for n in names]

metadata = {
    'protocolName': 'PCR Plate Prep with 384 Well Plate',
    'author': 'Rami Farawi <rami.farawi@opentrons.com>',
    'source': 'Custom Protocol Request',
    'apiLevel': '2.9'
}

def run(ctx: protocol_api.ProtocolContext):
    # noqa: F821
    [num_samp, overage_percent, mix_reps, p20_mount, p300_mount] = get_values("num_samp", "overage_percent", "mix_reps", "p20_mount", "p300_mount")

    if not 0 <= num_samp <= 384:
        raise Exception("Enter a sample number between 1-384")

    num_plates = math.ceil(num_samp/96)
    plates = [str(num+1) for num in range(0, num_plates)]
    overage = 1+overage_percent/100

    if not 0 <= num_samp <= 384:
        raise Exception("Enter a sample number between 1-384")

    # load labware
    sample_plates = [ctx.load_labware_from_definition(labware_def_n_2ml_96_wp, slot) for slot in plates]
    tuberack = ctx.load_labware_from_definition(labware_def_ot_24_tr_1500ul, '5')
    pcr_plate_384 = ctx.load_labware_from_definition(labware_def_pr_384_wp_50ul, '6')
    tiprack20 = [ctx.load_labware('opentrons_96_filtertiprack_20ul', slot) for slot in ['8', '9', '11']]
    tiprack300 = [ctx.load_labware('opentrons_96_tiprack_300ul', slot) for slot in ['7', '10']]

    # load instrument
    p300 = ctx.load_instrument('p300_multi_gen2', p300_mount, tip_racks=tiprack300)
    p20 = ctx.load_instrument('p20_single_gen2', p20_mount, tip_racks=tiprack20)

    num_channels_per_pickup = 1  # (only pickup tips on front-most channel)
    tips_ordered = [ 
        tip for rack in tiprack300 
        for row in rack.rows()[len(rack.rows())-num_channels_per_pickup::-1*num_channels_per_pickup]
        for tip in row]
    tip_count = 0

    def pick_up_300():
        nonlocal tip_count
        p300.pick_up_tip(tips_ordered[tip_count])
        tip_count += 1

    def pick_up_20():
        try:
            p20.pick_up_tip()
        except protocol_api.labware.OutOfTipsError:
            ctx.pause("Replace empty tip racks.")
            p20.reset_tipracks()
            pick_up_20()

    # protocol
    sample_wells = [well for plate in sample_plates for well in plate.wells()][:num_samp]

    # reagents
    total_mix_vol = 7*overage*num_samp
    num_mastermix_tubes = math.ceil((total_mix_vol)/1000)

    mastermix_tube = tuberack.rows()[0][:num_mastermix_tubes]
    positive_control = tuberack.rows()[3][5]

    liquid_prompt = f'''Please ensure you have: 
                    {total_mix_vol/num_mastermix_tubes}ul  of mastermix
                    in all tubes out of
                    {num_mastermix_tubes} mastermix tubes.'''
    ctx.comment(liquid_prompt)

    # plate mapping
    plate1_to_384 = [well for column in pcr_plate_384.columns()[::2] 
                     for well in column[::2]][:96 if 
                                              num_samp > 96 else num_samp]
    plate2_to_384 = [well for column in pcr_plate_384.columns()[1::2] 
                     for well in column[::2]][:96 if num_samp > 192 
                                              else num_samp-96 
                                              if num_samp-96 > 0 else 0]
    plate3_to_384 = [well for column in pcr_plate_384.columns()[::2] 
                     for well in column[1::2]][:96 if 
                                               num_samp > 288 
                                               else num_samp-192 
                                               if num_samp-192 > 0 else 0]
    plate4_to_384 = [well for column in pcr_plate_384.columns()[1::2] 
                     for well in column[1::2]][:num_samp-288 
                                               if num_samp-288 > 0 else 0]
    if num_samp == 384:
        plate4_to_384.pop()

    plates = {
                0: plate1_to_384,
                1: plate2_to_384,
                2: plate3_to_384,
                3: plate4_to_384
            }

    # distribute mastermix
    airgap = 5
    ctx.comment('Distributing Mastermix')
    pick_up_20()
    p20.aspirate(7, mastermix_tube[0].bottom(z=1.5))
    p20.air_gap(airgap)
    p20.dispense(7+airgap, pcr_plate_384.wells()[-1])

    for i, plate in enumerate(sample_plates):
        for source, well in zip(mastermix_tube*num_samp, plates[i]):

            p20.aspirate(7, source.bottom(z=1.5))
            p20.dispense(7+airgap, well)
            p20.blow_out()
    p20.drop_tip()
    ctx.comment('\n\n\n\n\n\n')

    # add positive control
    ctx.comment('Adding Positive Control')
    pick_up_20()
    p20.aspirate(5.5, positive_control.bottom(z=1.5))
    p20.air_gap(airgap)
    p20.dispense(7+airgap, pcr_plate_384.wells()[-1])
    p20.mix(mix_reps, 12.5, pcr_plate_384.wells()[-1])
    p20.drop_tip()

    # add sample and mix
    ctx.comment('Adding Sample')
    for i, plate in enumerate(sample_plates):
        for s, d in zip(sample_wells, plates[i]):
            pick_up_20()
            p20.aspirate(5.5, s.bottom(z=-2.75))
            p20.air_gap(airgap)
            p20.dispense(5.5+airgap, d)
            p20.mix(mix_reps, 12.5, d)
            p20.blow_out()
            p20.drop_tip()
        
run(ctx)
#for line in ctx.commands(): 
#    print(line)

C:\SPB_Data\.opentrons\robot_settings.json not found. Loading defaults
C:\SPB_Data\.opentrons\deck_calibration.json not found. Loading defaults


NameError: name 'protocol_api' is not defined

In [ ]:
# Original protocol of example from Problem Based Assessment
from opentrons import simulate
import numpy

# Initializing simulation to API version used
protocol = simulate.get_protocol_api('2.10')
# Homing simulated liquid handler machine
protocol.home()

# Labware
# magnetic plate and reservoir at slot 1
magdeck1 = protocol.load_module('magnetic module gen2', '1')
plate_reservoir = magdeck1.load_labware('nest_12_reservoir_15ml')
# magnetic module and well plate at slot 4
magdeck2 = protocol.load_module('magnetic module gen2', '4')
plate_well = magdeck2.load_labware('nest_96_wellplate_100ul_pcr_full_skirt')
# result well plate at slot 7
plate_result = protocol.load_labware('nest_96_wellplate_100ul_pcr_full_skirt', '7')
# pipette tip rack of 300uL at slot 5 and 6  
tiprack300 = [protocol.load_labware('opentrons_96_tiprack_300ul', slot) for slot in ['2']]
# pipette tip rack of 20uL at slot 2 and 3
tiprack20 = [protocol.load_labware('opentrons_96_tiprack_20ul', slot) for slot in ['3', '5', '6', '8', '9', '10', '11']]

# Pipettes
pip300multi = protocol.load_instrument('p300_multi_gen2', 'left', tip_racks=tiprack300)
pip20 = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=tiprack20)

# Programmable Values
bead_buffer_vol =  2000 #uL #must be < 15mL or draw from more reservoir wells
plate_well_vols = numpy.linspace(1.8, 18, num = 96) # can be read from file. # volumes for wells in order A1, B1, C1, ... to F12, G12, H12.
plate_well_vols_max = max(plate_well_vols)
plate_well_vols_sum = sum(plate_well_vols)

if (plate_well_vols_sum >= bead_buffer_vol):
    print("Error: Not enough buffer volume in bead mix to distribute to all wells")

# Commands
#1)	Pre-wash beads
for i in range(3): #1.g.	Repeat x3
    
    #1.a.	Add wash 
    #1.b.	Mix
    pip300multi.transfer(200, plate_reservoir.columns('2'), plate_reservoir.columns('1'), mix_after=(5, 100))

    #1.c.	activate magdeck 1
    #1.d.	Wait for pellet
    magdeck1.engage(height_from_base=4)
    protocol.delay(minutes=5)
    magdeck1.disengage()
        
    #1.e.	Remove old wash
    pip300multi.transfer(200, plate_reservoir.columns('1'), plate_reservoir.columns('12'))

#2)	Spread beads into wells
#2.a.	Add buffer to magdeck 1 for the highest needed concentration 
#2.b.	Mix
pip300multi.transfer(bead_buffer_vol, plate_reservoir.columns('6'), plate_reservoir.columns('1'), mix_after=(5, 100))

#2.c.	Place beads in wells at magdeck 2 at different volumes
pip20.pick_up_tip()
for i in range(96):
    pip20.transfer(plate_well_vols[i], plate_reservoir.columns('1'), plate_well.wells(i), new_tip='never')
pip20.drop_tip()

#2.d.	Top up buffer so all volumes are the same
#2.e.	Mix each one on magdeck 2
pip20.pick_up_tip()
for i in range(96):
    pip20.transfer((plate_well_vols_max - plate_well_vols[i]), plate_reservoir.columns('7'), plate_well.wells(i), mix_after=(3, int(plate_well_vols_max/2)),  new_tip='never')
pip20.drop_tip()

#3)	Incubate
#3.a.	Wait 30min 
protocol.delay(minutes=30)

#3.b.	Mix each well 
for i in range(96):
    pip20.transfer(10, plate_well.wells(i), plate_well.wells(i), mix_after=(3, int(plate_well_vols_max/2)))
    
#3.c.	Wait 30min 
protocol.delay(minutes=30)

#3.d.	Activate magdeck 2
magdeck2.engage()
protocol.delay(minutes=5)
magdeck2.disengage()

#3.e.	Remove supernatant in each well
pip20.pick_up_tip()
for i in range(96):
    pip20.transfer(plate_well_vols_max,  plate_well.wells(i), plate_reservoir.columns('12'), new_tip='never')
pip20.drop_tip()

#4)	Redo wash on magdeck 2
for j in range(3): #4.f.	Repeat x3

    #4.a.	Add wash to each well on magdeck 2
    #4.b.	Mix each well on magdeck 2
    for i in range(96):
        pip20.transfer(15, plate_reservoir.columns('3'), plate_well.wells(i), mix_after=(3, 10))

    #4.c.	Activate magdeck 2
    #4.d.	Wait for pellet on magdeck 2
    magdeck2.engage()
    protocol.delay(minutes=5)
    magdeck2.disengage()

    #4.e.	Remove old wash
    pip20.pick_up_tip()
    for i in range(96):
        pip20.transfer(15,  plate_well.wells(i), plate_reservoir.columns('12'),  new_tip='never')
    pip20.drop_tip()
    
#5)	Strip
#5.a.	Add acid to each well on magdeck 2
#5.b.	Mix each well
for i in range(96):
    pip20.transfer(15, plate_reservoir.columns('9'), plate_well.wells(i), mix_after=(5, 10))

#5.c.	Activate magdeck 2
magdeck2.engage()
protocol.delay(minutes=10)
magdeck2.disengage()

#5.d.	Take supernatant and move to fresh plate
for i in range(96):
    pip20.transfer(15, plate_well.wells(i), plate_result.wells(i))

    
for line in protocol.commands(): 
    print(line)

C:\SPB_Data\.opentrons\robot_settings.json not found. Loading defaults
C:\SPB_Data\.opentrons\deck_calibration.json not found. Loading defaults
This labware ({}) is not explicitly compatible with the Magnetic Module. You will have to specify a height when calling engage().
